In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time, datetime 

pd.set_option('display.max_columns', None)

# 定义数据库连接

In [44]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入订单数据

In [45]:
sql4 = '''
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,cc.name as channel_name         -- 来源渠道
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name
,topay.total_freeze_fund_amount 
,om.buy_service_product,tso.status as service_status 
,om.order_method
from  db_digua_business.t_order  om
left join db_digua_business.t_merchant tmer on tmer.id = om.merchant_id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
 -- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 
where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静'
,'陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静')
-- and COALESCE(pa.name, '未知') not in ("1000单秘密计划","1000单秘密计划-无优惠","1000单曙光计划","线下门店3个月试行") 
AND tmer.shop_type !=2  -- 剔除商家
and  DATE_FORMAT(om.create_time, '%Y-%m') >= DATE_FORMAT(DATE_ADD(CURRENT_DATE,INTERVAL -4 MONTH ), '%Y-%m')               -- 近15天数据

;
'''
df_order4 = query(sql4)
df_order4.shape

(462199, 29)

# 合并数据

In [46]:

df4 = df_order4.copy()
# df.order_id.notnull().sum()

In [47]:

df4["取消原因"]=df4["total_describes"].str.split("客户申请取消：").str[1].astype(str).str.split("$").str[0].str.strip()
df4["电审拒绝原因"]=df4["total_describes"].str.split("审核不通过：").str[1].astype(str).str.split("$").str[0].str.strip()
df4["无法联系原因"]=df4["total_describes"].str.split("用户无法联系：").str[1].astype(str).astype(str).str.split("$").str[0].str.strip()


df4.loc[:,"进件"]=np.where((df4["status2"]=="待支付")|(df4["status2"]=="订单取消"),"未进件","进件")
df4["进件"].value_counts()

# %%
df4.loc[:,"来源渠道"]=df4["channel_name"].fillna("未知渠道")

# 渠道归属

In [48]:
df4['拒绝理由'] = df4["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')
df4['拒绝理由'].value_counts()

拒绝理由
                                                                                      259081
冰鉴火眸分<500命中强拒                                                                          75855
命中强拒加入临时黑名单                                                                            24074
年龄超过49岁或低于18岁                                                                          12789
为法院限高被执行人                                                                               7612
                                                                                       ...  
命中青云分<500, 履约历史等级1, 履约历史等级=1强拒, 评分等级低于D,命中拒绝                                               1
命中策略240703_4强拒, 命中青云分<500, 履约历史等级1, 履约历史等级=1强拒                                             1
命中云商分低于515强拒(芝麻综合风险等级0豁免), 命中自有模型强拒, 蚁盾分>=80                                               1
命中云商分低于515强拒, 命中蚁盾分>50, 命中青云分<500, 蚁盾分>=80                                                 1
命中云商分低于515强拒(芝麻综合风险等级0豁免), 命中策略240703_4强拒, 命中融安分低于680强拒(芝麻综合风险等级0

In [49]:
def qudao_type(a,b,c):
    a = str(a)
    b = str(b)
    if "租物" in b:
        return "芝麻租物"
    elif "芝麻" in b:
        return "芝麻租物"
    elif "抖音" in b:
        return "抖音渠道"
    elif "搜索" in a:
        return "搜索渠道"
    elif "租物" in a:
        return "芝麻租物"
    elif "芝麻" in a:
        return "芝麻租物"
    elif c == 1:
        return "芝麻租物"   
    elif "叮咚直播" in a:
        return "叮咚直播"
    elif "租瓜直播2号" in a:
        return "租瓜直播2号"
    elif "租瓜直播" in a:
        return "租瓜直播"
    elif "直播" in a:
        return "小舞直播"
    elif "繁星" in a:
        return "繁星"
    elif "生活号" in a:
        return "生活号"
    elif "群" in a:
        return "支付宝社群"
    elif "付费灯火" in a:
        return "付费灯火"
    else :
        return a
    

# %%
df4.loc[:,"归属渠道"]=df4.apply(lambda x:qudao_type(x["来源渠道"],x["activity_name"],x["order_method"]),axis=1)
df4["归属渠道"].value_counts()

归属渠道
芝麻租物                   314661
搜索渠道                    74854
八派信息                    14771
单人聊天会话中的小程序消息卡片（分享）      8240
付费灯火                     7807
                        ...  
支付宝会员频道                     1
招商物料                        1
数立月活SL11                    1
深圳市新三友通信                    1
数立月活SL16                    1
Name: count, Length: 82, dtype: int64

In [50]:
# df4.columns

In [51]:
# 判断 前置拦截   机审强拒   出库前风控强拒
df4['是否前置拦截'] = np.where((df4.result.str.contains('id_card不得为空') )|(df4.result.str.contains('mobile校验不通过')) |(df4.result.str.contains('name校验不通过'))
                        |(df4.result.str.contains('年龄超过49岁或低于18岁'))|(df4.result=='风控未通过')|(df4.result=='黑名单用户'),1,0)
df4['是否机审强拒'] = np.where((df4.result.str.contains('风控拒绝') & (~df4.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
df4['是否出库前风控强拒'] = np.where((df4.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df4.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df4.result.str.contains('黑名单用户').fillna(False))) ,1,0)


In [52]:
def reject_type(a,b,c,d,e,f,g,h,i,j):
    if h==1 and b=="未进件":
        return "前置拦截"
    elif i==1 and b=="进件":
        return "机审强拒"    
    #抖音渠道
    # elif h==1 and b=="进件":
    #     return "机审强拒"  
    elif  b=="进件" and c is not np.nan:
        return "人审拒绝"
    elif  b=="进件" and d is not np.nan:
        return "客户取消"
    elif  b=="进件" and f is not np.nan:
        return "无法联系"
    elif  b=="进件" and j==1 :
        return "出库前风控强拒"
    elif  b=="进件" and "待审核" in e:
        return "待审核"
    elif  b=="进件" and "待发货" in e:
        return "出库"
    elif  b=="进件" and "待收货" in e:
        return "出库"
    elif b=="进件" and "租赁中" in e:
        return "出库"
    elif  b=="进件" and "已完成" in e:
        return "出库"
    elif a is np.nan and b=="未进件":
        return "未进件"
    elif  b=="未进件":
        return "进件前取消"
    else :
        return e

df4.loc[:,"审核状态"]=df4.apply(lambda x: reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)#,x["无法联系原因"]
df4["审核状态"].value_counts()  

审核状态
进件前取消    248412
前置拦截     151221
机审强拒      31565
人审拒绝      31001
Name: count, dtype: int64

In [53]:

df4_j = df4[df4["进件"]=="进件"]
df4_j.shape
df4["是否进件"] = np.where(df4["进件"]=='进件',1,0)

# 下单时间上月截止至今日的订单

## 到期订单

In [54]:
df4.columns

Index(['create_date', 'create_time', 'order_id', 'order_number', 'all_money',
       'status', 'status2', 'channel_type', 'sku_attributes', 'product_name',
       'new_actual_money', 'back_type', 'user_mobile', 'true_name',
       'id_card_num', 'total_describes', 'decision_result', 'cancel_reason',
       'trace_id', 'rejected', 'result', 'channel_name', 'activity_name',
       'merchant_id', 'merchant_name', 'total_freeze_fund_amount',
       'buy_service_product', 'service_status', 'order_method', '取消原因',
       '电审拒绝原因', '无法联系原因', '进件', '来源渠道', '拒绝理由', '归属渠道', '是否前置拦截', '是否机审强拒',
       '是否出库前风控强拒', '审核状态', '是否进件'],
      dtype='object')

In [57]:
df4['下单月份'] = df4['create_time'].dt.strftime('%Y-%m')
# df4['预计还款月份'] = df4['refund_date'].dt.strftime('%Y-%m')
create_month = df4['下单月份'].to_list()

In [66]:
Today1 = datetime.now().strftime('%Y-%m-%d')
def order_end(df, month1):
    # date_diff = (pd.to_datetime(Today1).to_period('M')-pd.to_datetime(month2).to_period('M')).n
    td = Today1
    df4_dq =df
    # if date_diff==0:
    #     td
    # elif date_diff==1:
    #     td = (pd.to_datetime(Today1)-timedelta(days=int(datetime.now().strftime('%d')))).strftime('%Y-%m-%d')
    # else:
    #     td = (pd.to_datetime(Today1)-timedelta(days=int(datetime.now().strftime('%d')))-pd.DateOffset(months=date_diff-1)).strftime('%Y-%m-%d')
    df4_dq = df[(df['下单月份']==month1)]
    # 月总订单
    dd_dq_month = df4_dq['order_number'].count()
    
    # 今日到期订单
    # df_dq_today = df4_dq[df4_dq['预计还款日期']==td]
    # dd_dq_today_count = df_dq_today['订单号'].count()
    # 今日到期已还款
    # dq_yhk = df_dq_today[df_dq_today['实付金额'] >= df_dq_today['应付金额']]
    # dd_dq_yhk = dq_yhk['订单号'].count()
    # 今日到期未还款
    # dq_whk = df_dq_today[df_dq_today['实付金额'] < df_dq_today['应付金额']]
    # dd_dq_whk = dq_whk['订单号'].count()
    # dd_dq_whk
    return df4_dq #,  df_dq_today, dq_yhk, dq_whk


In [59]:
def qd_info(df, qd, qd_name):    
    df = df[df[qd]==qd_name]
    df_count = df['订单号'].count()
    df_jj = df[df['是否进件']==1]
    df_jj = df_jj[['订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期', '实际还款日期', '应付金额', '实付金额']]
    df_jj_count = df_jj['订单号'].count()

    df_yq = df[df['逾期天数']>=1]
    df_yq = df_yq[['订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期', '实际还款日期', '逾期天数', '应付金额', '实付金额']]
    df_yq_count = df_yq['订单号'].count()
    df_yq
    return df_jj, df_yq

In [67]:
month1 = '2023-07'
month2 = '2024-07'
df4_dq = order_end(df4, month1)
df4_dq['下单月份'].drop_duplicates()

Series([], Name: 下单月份, dtype: object)

# 设置定时任务

In [ ]:
# from apscheduler.schedulers.blocking import BlockingScheduler
# from apscheduler.schedulers.background import BackgroundScheduler

# # hour = 9
# # minute = 10
# # week_name = pd.Timestamp.now().dayofweek
# H = datetime.now().strftime('%H')
# M = datetime.now().strftime('%M')
# Today1 = str(datetime.now().strftime('%Y%m%d%H'))
# def my_job1():
#     # print(f'执行定时任务：现在是每日的{hour}点{minute}分')
#     print(f'执行定时任务：现在是每日的{H}点{M}分')
#     path1 = 'F:/需求/到期订单/'
#     path2 = 'F:/需求/截止本月到期订单/'
#     path3 = 'F:/需求/八派、九州/'
#     # 到期提醒
#     with pd.ExcelWriter(path1 + f'到期订单_信审_{now_date}-{next_date}.xlsx', engine='openpyxl') as writer:
#         df_xs.to_excel(writer, sheet_name=now_date, index=False)
#     with pd.ExcelWriter(path1 + f'到期订单_催收_{now_date}-{next_date}.xlsx', engine='openpyxl') as writer:
#         df_cs.to_excel(writer, sheet_name=now_date, index=False)
#     # 本月到期订单与逾期订单
#     with pd.ExcelWriter(path2 + f'本月到期订单截止_{Today1}.xlsx', engine='openpyxl') as writer:
#         dd_all.to_excel(writer, sheet_name='订单汇总', index=False)
#     with pd.ExcelWriter(path2 + f'本月到期订单截止_{Today1}.xlsx', engine='openpyxl', mode='a') as writer:
#         df4_dq.to_excel(writer, sheet_name='本月到期订单', index=False)
#         df_dq_today.to_excel(writer, sheet_name='今日到期订单', index=False)
#         df_yq_1.to_excel(writer, sheet_name='逾期1+订单', index=False)
#         df_yq_4.to_excel(writer, sheet_name='逾期4+订单', index=False)
#         df_yq_7.to_excel(writer, sheet_name='逾期7+订单', index=False)
#         df_yq_15.to_excel(writer, sheet_name='逾期15+订单', index=False)
#         dq_yhk.to_excel(writer, sheet_name='今日到期已还款订单', index=False)
#         dq_whk.to_excel(writer, sheet_name='今日到期未还款订单', index=False)
#     # 渠道逾期数据
#     with pd.ExcelWriter(path3 + f'进件逾期数据_{Today1}.xlsx', engine='openpyxl') as writer:
#         df4_bp_jj.to_excel(writer, sheet_name='八派信息进件数据', index=False)
#     with pd.ExcelWriter(path3 + f'进件逾期数据_{Today1}.xlsx', engine='openpyxl', mode='a') as writer:
#         df4_bp_yq.to_excel(writer, sheet_name='八派信息逾期数据', index=False)
#         df4_jz_jj.to_excel(writer, sheet_name='九州信息进件数据', index=False)
#         df4_jz_yq.to_excel(writer, sheet_name='九州信息逾期数据', index=False)
# scheduler1 = BackgroundScheduler()
# # 添加一个cron任务，每天的16点55分执行
# # scheduler1.add_job(my_job1, 'cron', hour=hour, minute=minute)
# scheduler1.add_job(my_job1, 'cron', hour=H, minute=int(M)+1)

# # 启动scheduler1
# scheduler1.start()
    

# 到期订单（方全龙需求）

In [ ]:

# with pd.ExcelWriter(f'F:/需求/方全龙需求/订单_{dd_date}.xlsx', engine='openpyxl') as writer:
#     df5.to_excel(writer, sheet_name=f'订单_{dd_date}', index=False)